In [1]:
from main import AuditingFramework

## 创建对象

In [2]:
# 创建审计框架对象，确保运行过程中都在对同一个对象进行操作
fw = AuditingFramework()

## 基本信息

In [3]:
model = fw.get_model()
model.type

<bound method Module.type of MLP(
  (fc): ModuleList(
    (0): Linear(in_features=21, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (fcout): Linear(in_features=64, out_features=2, bias=True)
)>

In [4]:
fw.get_model_info()

{'name': 'MLP',
 'n_layers': 4,
 'hidden dimension': 64,
 'optimizer': 'Adam',
 'learning rate': 0.001}

In [5]:
fw.get_data_info()

{'name': 'adult',
 'task': 'Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset.',
 'columns': ['age',
  'capital-gain',
  'capital-loss',
  'education-num',
  'hours-per-week',
  'race_White',
  'sex_Male',
  'marital-status',
  'workclass'],
 'training set size': 36177,
 'testing set size': 9045}

In [6]:
fw.get_default_data_range()

{'age': [17, 90],
 'capital-gain': [0, 99999],
 'capital-loss': [0, 4356],
 'education-num': [1, 16],
 'hours-per-week': [1, 99],
 'race_White': [0, 1],
 'sex_Male': [0, 1],
 'marital-status': ['Divorced',
  'Married-AF-spouse',
  'Married-civ-spouse',
  'Married-spouse-absent',
  'Never-married',
  'Separated',
  'Widowed'],
 'workclass': ['Federal-gov',
  'Local-gov',
  'Private',
  'Self-emp-inc',
  'Self-emp-not-inc',
  'State-gov',
  'Without-pay']}

## 参数设定

In [7]:
# 设置敏感属性为“sex”或“race”
fw.set_sensitive_attr('race')
fw.dataset.dim_feature() # 设置敏感属性后会初始化数据集、数据加载等相关属性，这里举例展示数据维度

21

In [8]:
# 设置数据范围
user_selected_data_range = {
    'age': [17, 90], # 默认范围以内的任意闭区间
    'capital-gain': [0, 99999], # 默认范围以内的任意闭区间
    'capital-loss': [0, 4356], # 默认范围以内的任意闭区间
    'education-num': [1, 16], # 默认范围以内的任意闭区间
    'hours-per-week': [1, 99], # 默认范围以内的任意闭区间
    'race_White': [0, 1], # 默认范围的任意非空子集： [0]（只有女性）, [1]（只有男性）, [0, 1]（男性 & 女性）
    'sex_Male': [0, 1], # 默认范围的任意非空子集： [0]（只有非白人），[1]（只有白人），[0, 1]（白人 & 非白人）
    'marital-status': [
        'Divorced',
        'Married-AF-spouse',
        'Married-civ-spouse',
        'Married-spouse-absent',
        'Never-married',
        'Separated',
        'Widowed'
    ], # 默认范围的任意非空子集：如['Divorced'], 或['Married-civ-spouse', 'Married-spouse-absent', 'Never-married']
    'workclass': [
        'Federal-gov',
        'Local-gov',
        'Private',
        'Self-emp-inc',
        'Self-emp-not-inc',
        'State-gov',
        'Without-pay'
    ] # 默认范围的任意非空子集：如['Self-emp-inc', 'Self-emp-not-inc']，或['Local-gov', 'Private']
}
fw.set_data_range(user_selected_data_range)
fw.get_data_range()

{'age': [17, 90],
 'capital-gain': [0, 99999],
 'capital-loss': [0, 4356],
 'education-num': [1, 16],
 'hours-per-week': [1, 99],
 'race_White': [0, 1],
 'sex_Male': [0, 1],
 'marital-status': ['Divorced',
  'Married-AF-spouse',
  'Married-civ-spouse',
  'Married-spouse-absent',
  'Never-married',
  'Separated',
  'Widowed'],
 'workclass': ['Federal-gov',
  'Local-gov',
  'Private',
  'Self-emp-inc',
  'Self-emp-not-inc',
  'State-gov',
  'Without-pay']}

In [9]:
# 指定个体公平指标

default_eps = fw.get_default_eps() # 默认epsilon值：1e8
# 设置个体公平的评价准则，包含两个参数，dx=‘LR’或‘Eu’，eps是一个浮点数。
fw.set_individual_fairness_metric(dx='LR', eps=default_eps)

## 访问模型

In [10]:
# 随机构造一个样本访问模型
random_sample = {
    'age': 53,
    'capital-gain': 76,
    'capital-loss': 385,
    'education-num': 3,
    'hours-per-week': 31,
    'race_White': 'White',
    'sex_Male': 'Female',
    'marital-status': 'Widowed',
    'workclass': 'Self-emp-inc'
}
fw.query_model(random_sample) # 输出模型将其分类为1的概率

0.011759641580283642

In [11]:
# 以上述随机构造的样本为起始点搜索成对不公平样本
unfair_pair = fw.seek_unfair_pair(random_sample)
unfair_pair

[{'age': 53,
  'capital-gain': 4617,
  'capital-loss': 483,
  'education-num': 3,
  'hours-per-week': 31,
  'race_White': 'Others',
  'sex_Male': 'Female',
  'marital-status': 'Never-married',
  'workclass': 'Private'},
 {'age': 53,
  'capital-gain': 4617,
  'capital-loss': 483,
  'education-num': 3,
  'hours-per-week': 31,
  'race_White': 'White',
  'sex_Male': 'Female',
  'marital-status': 'Never-married',
  'workclass': 'Private'}]

In [12]:
fw.get_unfair_pair()

[{'age': 53,
  'capital-gain': 4617,
  'capital-loss': 483,
  'education-num': 3,
  'hours-per-week': 31,
  'race_White': 'Others',
  'sex_Male': 'Female',
  'marital-status': 'Never-married',
  'workclass': 'Private'},
 {'age': 53,
  'capital-gain': 4617,
  'capital-loss': 483,
  'education-num': 3,
  'hours-per-week': 31,
  'race_White': 'White',
  'sex_Male': 'Female',
  'marital-status': 'Never-married',
  'workclass': 'Private'}]

In [13]:
# 模型优化：提供给模型找到的不公平样本
# new_model = fw.optimize(fw.model, unfair_pair=unfair_pair)

## 指标

In [14]:
fw.accuracy(fw.model)

tensor(0.8387)

In [15]:
fw.group_fairness_metric(fw.model)

0.4177777777777778

In [16]:
fw.local_individual_fairmess_metric(fw.model, unfair_pair[0])

2.891179323196411

In [17]:
# 初始模型的global_individual_fairness = 0.6773，这个指标计算特别慢，可以直接用这个数字
# fw.global_individual_fairness_metric(fw.model)

In [18]:
# 找到的样本在老模型中得到了不公平的对待
fw.fair(fw.model, unfair_pair)

(False, 0.0, 1.0)

In [19]:
# 但是在新模型中被公平对待
# fw.fair(new_model, unfair_pair)